In [1]:
!pip install langchain langchain_community sentence-transformers datasets faiss-cpu torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Импортирую необходимые библиотеки

In [2]:
!pip install -U langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 11.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1


In [3]:
from datasets import load_dataset
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.llms.base import LLM
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os
from accelerate import Accelerator 
from datasets import concatenate_datasets

# Для удобства ввожу глобальные переменные (здесь я использую набор данных juliet_faiss_index и языковую модель qwen )

In [4]:
class GlobalParameters:
    def __init__(self):
        self.emb_name = 'sentence-transformers/all-MiniLM-L6-v2'
        self.vector_data_path = 'juliet_faiss_index'
        self.model_name = "Qwen/Qwen2.5-3B-Instruct"

gb = GlobalParameters()

dataset = load_dataset("LorenzH/juliet_test_suite_c_1_3")

dataset

README.md: 0.00B [00:00, ?B/s]

jts_c_1_3_train.csv:   0%|          | 0.00/237M [00:00<?, ?B/s]

jts_c_1_3_test.csv:   0%|          | 0.00/59.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/80706 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20177 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'filename', 'class', 'good', 'bad'],
        num_rows: 80706
    })
    test: Dataset({
        features: ['index', 'filename', 'class', 'good', 'bad'],
        num_rows: 20177
    })
})

In [5]:
dataset['train'][100]

{'index': 53674,
 'filename': 'testcases/CWE190_Integer_Overflow/s07/CWE190_Integer_Overflow__char_rand_preinc_21.c',
 'class': 2,
 'good': '/* TEMPLATE GENERATED TESTCASE FILE\nFilename: CWE190_Integer_Overflow__char_rand_preinc_21.c\nLabel Definition File: CWE190_Integer_Overflow.label.xml\nTemplate File: sources-sinks-21.tmpl.c\n*/\n/*\n * @description\n * CWE: 190 Integer Overflow\n * BadSource: rand Set data to result of rand()\n * GoodSource: Set data to a small, non-zero number (two)\n * Sinks: increment\n *    GoodSink: Ensure there will not be an overflow before incrementing data\n *    BadSink : Increment data, which can cause an overflow\n * Flow Variant: 21 Control flow: Flow controlled by value of a static global variable. All functions contained in one file.\n *\n * */\n\n#include "std_testcase.h"\n\n\n    return 0;\n}\n\n#endif\n',
 'bad': '/* TEMPLATE GENERATED TESTCASE FILE\nFilename: CWE190_Integer_Overflow__char_rand_preinc_21.c\nLabel Definition File: CWE190_Integer

In [6]:
dataset['test'][100]

{'index': 491,
 'filename': 'testcases/CWE404_Improper_Resource_Shutdown/CWE404_Improper_Resource_Shutdown__freopen_w32CloseHandle_06.c',
 'class': 181,
 'good': '/* TEMPLATE GENERATED TESTCASE FILE\nFilename: CWE404_Improper_Resource_Shutdown__freopen_w32CloseHandle_06.c\nLabel Definition File: CWE404_Improper_Resource_Shutdown.label.xml\nTemplate File: source-sinks-06.tmpl.c\n*/\n/*\n * @description\n * CWE: 404 Improper Resource Shutdown or Release\n * BadSource: freopen Open a file using freopen()\n * Sinks: w32CloseHandle\n *    GoodSink: Close the file using fclose()\n *    BadSink : Close the file using CloseHandle\n * Flow Variant: 06 Control flow: if(STATIC_CONST_FIVE==5) and if(STATIC_CONST_FIVE!=5)\n *\n * */\n\n#include "std_testcase.h"\n\n#include <windows.h>\n\n/* The variable below is declared "const", so a tool should be able\n   to identify that reads of this will always give its initialized\n   value. */\nstatic const int STATIC_CONST_FIVE = 5;\n\n\n    return 0;\n}\n

In [7]:
#dataset = concatenate_datasets([dataset['train'], dataset['test']])
dataset_train = dataset['train']

In [8]:
import re

def preproccesing_data(dataset):

    cpp_text = []
    metadata = []

    for i, item in enumerate(dataset):
    
        def extract_description(code):
            start = code.find('@description')
            end = code.find('*/', start)
            return code[start+13:end].strip() if start != -1 else ""
    
        def extract_cwe(text):
            """
            Извлекает CWE из description. Если не найдено, возвращает None.
            """
            match = re.search(r"\* CWE:\s*(CWE[-\d]+|\d+)", text, re.IGNORECASE)
            if match:
                # Если найдено только число, добавляем CWE-
                cwe = match.group(1)
                if cwe.isdigit():
                    return f"CWE-{cwe}"
                else:
                    return cwe.upper()
            else:
                return None
    
        def extract_code(code):
            include_pos = code.find('#include')
            if include_pos != -1:
                return code[include_pos:].strip()
            return code.strip()
    
        if item.get('good'):
            description = extract_description(item['good'])
            code_clean = extract_code(item['good'])
    
            cpp_text.append(code_clean)
            metadata.append({
                'type': 'good',
                'id': item['index'],
                'cwe': None,  
                'description': description,
                'code': code_clean
            })
    
        if item.get('bad'):
            description = extract_description(item['bad'])
            cwe_id = extract_cwe(description)
            code_clean = extract_code(item['bad'])
    
            cpp_text.append(code_clean)
            metadata.append({
                'type': 'bad',
                'id': item['index'],
                'cwe': cwe_id,
                'description': description,
                'code': code_clean
            })

    return metadata

metadata = preproccesing_data(dataset_train)
metadata[:5]

[{'type': 'good',
  'id': 29645,
  'cwe': None,
  'description': '* CWE: 126 Buffer Over-read\n * BadSource:  Use a small buffer\n * GoodSource: Use a large buffer\n * Sink: loop\n *    BadSink : Copy data to string using a loop\n * Flow Variant: 53 Data flow: data passed as an argument from one function through two others to a fourth; all four functions are in different source files\n *\n *',
  'code': '#include "std_testcase.h"\n\n#include <wchar.h>\n\n/* all the sinks are the same, we just want to know where the hit originated if a tool flags one */\n\n\n\n#ifndef OMITGOOD\n\n/* good function declaration */\nvoid CWE126_Buffer_Overread__malloc_char_loop_53d_goodG2BSink(char * data);\n\n/* goodG2B uses the GoodSource with the BadSink */\nvoid CWE126_Buffer_Overread__malloc_char_loop_53c_goodG2BSink(char * data)\n{\n    CWE126_Buffer_Overread__malloc_char_loop_53d_goodG2BSink(data);\n}\n\n#endif /* OMITGOOD */'},
 {'type': 'bad',
  'id': 29645,
  'cwe': 'CWE-126',
  'description': '* 

In [9]:
len(metadata)

161412

In [10]:
from langchain.schema import Document

docs = [
    Document(
        page_content=f"Description: {m['description']}\nCode:\n{m['code']}",
        metadata={
            "id": m['id'],
            "type": m['type'],
            "cwe": m['cwe'],
            "description": m['description'],
            "code": m['code']
        }
    )
    for m in metadata
]

In [11]:
embedding_model = HuggingFaceEmbeddings(
    model_name=gb.emb_name, multi_process=True, 
    encode_kwargs={'normolize_embeddings': True})

if not os.path.exists(gb.vector_data_path):
    vector_db = FAISS.from_documents(
        documents=docs,
        embedding=embedding_model
    )
    vector_db.save_local(gb.vector_data_path)
else:
    vector_db = FAISS.load_local(
        folder = gb.vector_data_path,
        embedding=embedding_model,
        allow_dangerous_deserialization=True
    )

2025-08-28 09:45:35.824978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374336.016406      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374336.071222      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-08-28 09:46:04.111202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374364.134877     158 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374364.142100     158 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 09:46:13.820344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374373.843809     170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374373.851015     170 cuda_blas.cc:1

Chunks:   0%|          | 0/33 [00:00<?, ?it/s]

In [12]:
!pip install transformers-stream-generator

  Preparing metadata (setup.py) ... done
  Created wheel for transformers-stream-generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=5c7268f2c4a3dcfc64e5b09a88e12fa4c7c5da3f59665c56772f7549dccce47e
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers-stream-generator


In [13]:
tokenizer = AutoTokenizer.from_pretrained(gb.model_name, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(gb.model_name, trust_remote_code=True)

accelerator = Accelerator()
model = accelerator.prepare(model)

def generate_qwen(promt, max_tokens=648):
    inputs = tokenizer(promt, return_tensors='pt').to(accelerator.device)
    with accelerator.autocast():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            #temperature=0.7,
            do_sample=False,    
            #top_p=0.9 
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [14]:

prompt_template = """
Ты — эксперт по безопасности C/C++ с глубоким знанием уязвимостей CWE.
Используй контекст ({context}: description, cwe, code) для анализа предоставленного кода {question}.

Если в коде есть уязвимость, выведи строго в формате:

cwe: CWE-XXX
fixed:
<многострочный исправленный код, исправляющий уязвимость>

Если уязвимости нет, выведи:

cwe: CWE-None
fixed: None

Важные правила:
- Выводи только один CWE за один вызов.
- Код fixed может быть многострочным.
- Никаких дополнительных комментариев, рассуждений, заголовков или контекста.
"""



# prompt_template = """
# Ты — опытный эксперт по информационной безопасности (C/C++), специализирующийся на анализе уязвимостей на основе CWE.
# Используй ВСЮ информацию контекста {context} description, cwe, code, чтобы определить CWE в предоставленном коде {question}.

# Если есть CWE уязвимость ответь строго в формате:
# cwe: CWE-XXX

# Если CWE НЕ была обнаружена ответь строго в формате:
# cwe: CWE-None

# Ты должен вывести строго одну строку по шаблону и больше ничего! Не печатай context и question.
# """




PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question']
)


In [15]:
class QwenLLM(LLM):
    def _call(self, promt, stop=None):
        return generate_qwen(promt)
    #свойства для LangChain

    @property
    def _identifying_params(self):
        return {'name': 'Qwen'}

    @property
    def _llm_type(self):
        return 'qwen'

qwen_llm = QwenLLM()

In [16]:
qa_chain = RetrievalQA.from_llm(
    llm = qwen_llm,
    retriever = vector_db.as_retriever(search_kwargs={'k':3}),
    return_source_documents = False
)

qa_chain.combine_documents_chain.llm_chain.prompt = PROMPT


import re

def get_qa_answer(cpp_code, chain=qa_chain, use_penultimate=False):

    result = chain.invoke({'query': cpp_code})
    text = result['result']

    # Находим все блоки cwe + fixed
    matches = re.findall(r'cwe:\s*(CWE-\d+|CWE-None)\s*fixed:\s*(.+?)(?=cwe:|$)', text, re.DOTALL | re.IGNORECASE)

    if not matches:
        return {'cwe': 'CWE-None', 'fixed': 'None'}

    # Выбираем последний или предпоследний блок
    index = -2 if use_penultimate and len(matches) >= 2 else -1
    cwe, fixed = matches[index]
    fixed = fixed.strip()

    return {'cwe': cwe, 'fixed': fixed}


In [17]:
import torch
torch.cuda.empty_cache()

cpp_code_false="""
#include "std_testcase.h"

#ifndef OMITBAD

void CWE121_Stack_Based_Buffer_Overflow__CWE805_int64_t_declare_memmove_02_bad()
{
    int64_t * data;
    int64_t dataBadBuffer[50];
    int64_t dataGoodBuffer[100];

    if(1)
    {
        /* FLAW: Set a pointer to a "small" buffer. This buffer will be used in the sinks as a destination
         * buffer in various memory copying functions using a "large" source buffer. */
        data = dataBadBuffer;
    }

    {
        int64_t source[100] = {0}; /* fill with 0's */
        /* POTENTIAL FLAW: Possible buffer overflow if data < 100 */
        memmove(data, source, 100 * sizeof(int64_t));
        printLongLongLine(data[0]);
    }
}

#endif /* OMITBAD */

int main()
{
#ifndef OMITBAD
    printLine("Calling bad()...");
    CWE121_Stack_Based_Buffer_Overflow__CWE805_int64_t_declare_memmove_02_bad();
    printLine("Finished bad()");
#endif /* OMITBAD */

    return 0;
}

"""




answer = get_qa_answer(cpp_code_false)
print(answer)


2025-08-28 09:51:24.313674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374684.339404     286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374684.347181     286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 09:51:34.430046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374694.454884     298 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374694.462331     298 cuda_blas.cc:1

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{'cwe': 'CWE-121', 'fixed': '#include "std_testcase.h"\n\n#ifndef OMITBAD\n\nvoid CWE121_Stack_Based_Buffer_Overflow__CWE805_int64_t_declare_memmove_02_bad()\n{\n    int64_t * data;\n    int64_t dataBadBuffer[50];\n    int64_t dataGoodBuffer[100];\n\n    if(1)\n    {\n        /* FLAW: Set a pointer to a "small" buffer. This buffer will be used in the sinks as a destination\n         * buffer in various memory copying functions using a "large" source buffer. */\n        data = dataBadBuffer;\n    }\n\n    {\n        int64_t source[100] = {0}; /* fill with 0\'s */\n        /* FIX: Set the size of the destination buffer to the size of the source buffer */\n        size_t sourceSize = sizeof(source);\n        /* POTENTIAL FLAW: Possible buffer overflow if sourceSize if larger than the destination buffer */\n        memmove(data, source, sourceSize * sizeof(int64_t));\n        printLongLongLine(data[0]);\n    }\n}\n\n#endif /* OMITBAD */\n\nint main()\n{\n#ifndef OMITBAD\n    printLine("Cal

In [18]:
dataset_test = dataset['test']
metadata = preproccesing_data(dataset_test)

from collections import Counter

cwe_list = [m['cwe'] for m in metadata if m['cwe'] is not None]

# считаем количество каждого CWE
cwe_counts = Counter(cwe_list)

# выводим
for cwe, count in cwe_counts.most_common():
    print(f"{cwe}: {count}")

# если хочешь, можно узнать общее количество классов
print(f"\nNumber of classes: {len(cwe_counts)}")


CWE-122: 1934
CWE-121: 1556
CWE-78: 1548
CWE-190: 1266
CWE-762: 1250
CWE-134: 955
CWE-191: 945
CWE-590: 877
CWE-23: 809
CWE-36: 715
CWE-124: 688
CWE-127: 642
CWE-401: 530
CWE-126: 496
CWE-194: 389
CWE-195: 387
CWE-789: 345
CWE-415: 338
CWE-690: 303
CWE-369: 301
CWE-197: 261
CWE-400: 230
CWE-457: 229
CWE-761: 190
CWE-114: 177
CWE-680: 170
CWE-563: 161
CWE-427: 161
CWE-606: 155
CWE-404: 131
CWE-90: 131
CWE-416: 124
CWE-253: 122
CWE-252: 118
CWE-758: 113
CWE-476: 105
CWE-617: 103
CWE-426: 73
CWE-675: 68
CWE-319: 62
CWE-665: 57
CWE-773: 50
CWE-123: 49
CWE-775: 48
CWE-272: 48
CWE-398: 45
CWE-259: 45
CWE-284: 39
CWE-588: 36
CWE-321: 35
CWE-256: 33
CWE-591: 29
CWE-506: 27
CWE-843: 22
CWE-666: 22
CWE-390: 21
CWE-377: 20
CWE-176: 15
CWE-244: 15
CWE-15: 15
CWE-510: 14
CWE-672: 14
CWE-511: 14
CWE-226: 13
CWE-464: 13
CWE-467: 13
CWE-188: 11
CWE-328: 11
CWE-546: 11
CWE-475: 11
CWE-391: 11
CWE-468: 10
CWE-459: 10
CWE-681: 9
CWE-535: 8
CWE-396: 8
CWE-325: 8
CWE-367: 8
CWE-667: 7
CWE-327: 7
CWE-366: 7

In [19]:
metadata[1]

{'type': 'bad',
 'id': 3,
 'cwe': 'CWE-591',
 'description': '* CWE: 591 Sensitive Data Storage in Improperly Locked Memory\n * BadSource:  Allocate memory for sensitive data without using VirtualLock() to lock the buffer into memory\n * GoodSource: Allocate memory for sensitive data and use VirtualLock() to lock the buffer into memory\n * Sink:\n *    BadSink : Authenticate the user using LogonUserA()\n * Flow Variant: 53 Data flow: data passed as an argument from one function through two others to a fourth; all four functions are in different source files\n *\n *',
 'code': '#include "std_testcase.h"\n\n#include <wchar.h>\n#include <windows.h>\n\n#pragma comment(lib, "advapi32.lib")\n\n/* all the sinks are the same, we just want to know where the hit originated if a tool flags one */\n\n#ifndef OMITBAD\n\n/* bad function declaration */\nvoid CWE591_Sensitive_Data_Storage_in_Improperly_Locked_Memory__w32_char_53d_badSink(char * password);\n\nvoid CWE591_Sensitive_Data_Storage_in_Impro

In [20]:
# import random
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# import os

# random.seed(42)
# shuffled_metadata = metadata.copy()
# random.shuffle(shuffled_metadata)

# subset = [m for m in shuffled_metadata if m['cwe'] is not None][:200]

# y_true, y_pred = [], []

# for item in subset:
#     code_snippet = item["code"]
#     true_cwe = item["cwe"]

#     answer = get_qa_answer(code_snippet)

#     predicted_cwes = extract_cwes(answer)
#     pred_cwe = predicted_cwes[0] if predicted_cwes else "CWE-NONE"

#     y_true.append(true_cwe)
#     y_pred.append(pred_cwe)
#     print(f"True: {true_cwe} | Pred: {pred_cwe}")

# print("Accuracy:", accuracy_score(y_true, y_pred))
# print("Precision (macro):", precision_score(y_true, y_pred, average='macro', zero_division=0))
# print("Recall (macro):", recall_score(y_true, y_pred, average='macro', zero_division=0))
# print("F1 (macro):", f1_score(y_true, y_pred, average='macro', zero_division=0))

# print("\nDetailed classification report:")
# print(classification_report(y_true, y_pred, zero_division=0))


In [21]:
!pip install codebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 kB 10.6 MB/s eta 0:00:00a 0:00:01


In [22]:
import random
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from codebleu import calc_codebleu

random.seed(42)

shuffled_metadata = metadata.copy()
random.shuffle(shuffled_metadata)

bad_items = [m for m in shuffled_metadata if m['type'] == 'bad'][:100]
good_dict = {m['id']: m for m in metadata if m['type'] == 'good'}

y_true, y_pred = [], []
refs, hyps = [], []

for bad in bad_items:
    code_snippet = bad['code']
    true_cwe = bad['cwe']

    answer = get_qa_answer(code_snippet)  
    pred_cwe = answer['cwe']
    fixed_code = answer['fixed']

    good_code = good_dict.get(bad['id'], {}).get('code', None)

    if good_code and fixed_code and fixed_code != "None":
        refs.append([good_code])
        hyps.append(fixed_code)

    y_true.append(true_cwe)
    y_pred.append(pred_cwe)

    print(f"True: {true_cwe} | Pred: {pred_cwe} | Fixed snippet: {fixed_code[:60]}...")


print("\n Finish")

2025-08-28 09:52:42.636711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374762.661288     328 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374762.668545     328 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 09:52:52.642385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374772.666269     340 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374772.673710     340 cuda_blas.cc:1

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


True: CWE-191 | Pred: CWE-191 | Fixed snippet: void CWE191_Integer_Underflow__short_fscanf_sub_64_bad()
{
 ...


2025-08-28 09:53:21.310519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374801.334519     360 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374801.342126     360 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 09:53:31.344855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756374811.370454     372 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756374811.378213     372 cuda_blas.cc:1

...

True: CWE-476 | Pred: CWE-476 | Fixed snippet: void bad()
{
    char * data;
    list<char *> dataList;
   ...


2025-08-28 11:32:56.667993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756380776.692542    3498 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756380776.699951    3498 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 11:33:06.573109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756380786.597618    3510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756380786.604874    3510 cuda_blas.cc:1

Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


True: CWE-400 | Pred: CWE-400 | Fixed snippet: #ifndef OMITBAD
    void CWE400_Resource_Exhaustion__rand_fo...

 Finish


In [27]:
print("Количество None в y_true:", sum(1 for x in y_true if x is None))
print("Количество None в y_pred:", sum(1 for x in y_pred if x is None))


Количество None в y_true: 1
Количество None в y_pred: 0


In [28]:
filtered_pairs = [(true, pred) for true, pred in zip(y_true, y_pred) if true is not None and pred is not None]
y_true_filtered, y_pred_filtered = zip(*filtered_pairs)

print("Accuracy:", accuracy_score(y_true_filtered, y_pred_filtered))
print("Precision (macro):", precision_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))
print("Recall (macro):", recall_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))
print("F1 (macro):", f1_score(y_true_filtered, y_pred_filtered, average='macro', zero_division=0))


Accuracy: 0.7474747474747475
Precision (macro): 0.7952380952380952
Recall (macro): 0.70426097711812
F1 (macro): 0.7353741496598639


In [31]:
!pip install tree-sitter==0.20.0
!pip install tree-sitter-cpp==0.21.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.9/462.9 kB 8.6 MB/s eta 0:00:00ta 0:00:01


In [32]:

filtered_pairs = [(ref, hyp) for ref, hyp in zip(refs, hyps) if ref and hyp and hyp != "None"]
if not filtered_pairs:
    print("CodeBLEU Score: нет данных для оценки")
else:
    refs_filtered, hyps_filtered = zip(*filtered_pairs)
    
    try:
        lang = "cpp" 
        codebleu_score = calc_codebleu(list(refs_filtered), list(hyps_filtered), lang)
        print("\n=== CodeBLEU Score для всего корпуса ===")
        print(codebleu_score)
    except Exception as e:
        print(f"\nCodeBLEU Score не удалось посчитать: {e}")



=== CodeBLEU Score для всего корпуса ===
{'codebleu': 0.4879070085340173, 'ngram_match_score': 0.2578170959608222, 'weighted_ngram_match_score': 0.46514655552747336, 'syntax_match_score': 0.5954705193283873, 'dataflow_match_score': 0.6331938633193863}
